In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from pycaret.classification import *
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
import time

In [51]:
pd.set_option('display.max_columns', None)
participants_train = pd.read_csv('../data/participants_train.csv') 
participants_test = pd.read_csv('../data/participants_test.csv') 
winner_train = pd.read_csv('../data/train_winners.csv')
champion_mastery = pd.read_csv('../data/champion_mastery.csv')
#submission = pd.read_csv('data/sample_submission.csv')
match_dates = pd.read_csv('../data/matchdates.csv')
patch13_5 = pd.read_csv('../data/13_5.csv') # between 2023-03-08 to 2023-03-22
patch13_4 = pd.read_csv('../data/13_4.csv') # between 2023-02-23 to 2023-03-07
patch13_3 = pd.read_csv('../data/13_3.csv') # between 2023-02-08 to 2023-02-22
patch13_1 = pd.read_csv('../data/13_1.csv') # between 2023-01-11 to 2023-02-07
#team_train = pd.read_csv('data/team_trained.csv')
#team_test = pd.read_csv('data/team_test.csv')

In [29]:
# create a dictionary of name mappings
name_mapping = {
    "Kaisa": "Kai'Sa",
    "Nunu": "Nunu & Willump",
    "Renata": "Renata Glasc",
    "MonkeyKing": "Wukong",
    "Leblanc": "LeBlanc",
    "Velkoz": "Vel'Koz",
    "Belveth": "Bel'Veth",
    "FiddleSticks": "Fiddlesticks",
    "Khazix": "Kha'Zix",
    "Chogath": "Cho'Gath",
}

# convert the dictionary to a list
name_list = list(name_mapping.items())

# replace the values using the name_list
participants_train['championName'] = participants_train['championName'].replace(dict(name_list))
participants_test['championName'] = participants_test['championName'].replace(dict(name_list))

In [30]:
participants_train =  pd.merge(participants_train, champion_mastery, on=['championId', 'summonerId'])
participants_train =  pd.merge(participants_train, match_dates, on=['matchId'])
participants_train

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate
0,0,100,1,0,303,Mordekaiser,82,6,61890,True,3,2023-03-11
1,0,100,2,1,616,Sylas,517,5,37664,True,2,2023-03-11
2,0,100,3,2,667,Lissandra,127,6,40303,True,1,2023-03-11
3,0,100,4,3,860,Caitlyn,51,6,96304,True,3,2023-03-11
4,0,100,5,4,325,Morgana,25,7,608721,True,0,2023-03-11
...,...,...,...,...,...,...,...,...,...,...,...,...
79983,7978,200,6,39592,630,Renekton,58,7,65103,False,0,2023-03-11
79984,7978,200,7,39593,232,Gragas,79,7,185423,True,0,2023-03-11
79985,7978,200,8,39594,431,Irelia,39,6,39957,False,2,2023-03-11
79986,7978,200,9,4383,309,Xayah,498,5,25070,False,1,2023-03-11


In [52]:
participants_test =  pd.merge(participants_test, champion_mastery, on=['championId', 'summonerId'])
participants_test =  pd.merge(participants_test, match_dates, on=['matchId'])
participants_test

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate
0,8000,100,1,23938,57,Fiora,114,3,10896,True,0,2023-03-12
1,8000,100,2,8037,788,LeeSin,64,7,143994,False,0,2023-03-12
2,8000,100,3,8444,566,Veigar,45,7,203012,True,0,2023-03-12
3,8000,100,4,24831,542,Twitch,29,7,376388,True,0,2023-03-12
4,8000,100,5,23890,224,Soraka,16,7,625009,True,0,2023-03-12
...,...,...,...,...,...,...,...,...,...,...,...,...
19993,9999,200,6,45320,458,Mordekaiser,82,7,117779,True,0,2023-03-11
19994,9999,200,7,45321,330,FiddleSticks,9,5,22922,False,0,2023-03-11
19995,9999,200,8,45322,462,Tristana,18,7,169934,False,0,2023-03-11
19996,9999,200,9,45323,256,Samira,360,4,19716,True,0,2023-03-11


In [32]:
# Merge patch13_1 onto participants_train using championName and matchcreationDate condition
participants_train = pd.merge(participants_train, patch13_1, on=['championName'], how='left')
participants_train.loc[(participants_train['matchcreationDate'] >= '2023-01-18') & (participants_train['matchcreationDate'] <= '2023-02-07'), '13_1_winrate'] = participants_train['win_rate']
participants_train.drop(columns=['win_rate'], inplace=True)

# Merge patch13_3 onto participants_train using championName and matchcreationDate condition
participants_train = pd.merge(participants_train, patch13_3, on=['championName'], how='left')
participants_train.loc[(participants_train['matchcreationDate'] >= '2023-02-08') & (participants_train['matchcreationDate'] <= '2023-02-22'), '13_3_winrate'] = participants_train['win_rate']
participants_train.drop(columns=['win_rate'], inplace=True)

# Merge patch13_4 onto participants_train using championName and matchcreationDate condition
participants_train = pd.merge(participants_train, patch13_4, on=['championName'], how='left')
participants_train.loc[(participants_train['matchcreationDate'] >= '2023-02-23') & (participants_train['matchcreationDate'] <= '2023-03-07'), '13_4_winrate'] = participants_train['win_rate']
participants_train.drop(columns=['win_rate'], inplace=True)

# Merge patch13_5 onto participants_train using championName and matchcreationDate condition
participants_train = pd.merge(participants_train, patch13_5, on=['championName'], how='left')
participants_train.loc[(participants_train['matchcreationDate'] >= '2023-03-08') & (participants_train['matchcreationDate'] <= '2023-03-22'), '13_5_winrate'] = participants_train['win_rate']
participants_train.drop(columns=['win_rate'], inplace=True)
participants_train


,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate,13_1_winrate,13_3_winrate,13_4_winrate,13_5_winrate
0,0,100,1,0,303,Mordekaiser,82,6,61890,True,3,2023-03-11,NaN,NaN,NaN,0.525100
1,0,100,2,1,616,Sylas,517,5,37664,True,2,2023-03-11,NaN,NaN,NaN,0.490833
2,0,100,3,2,667,Lissandra,127,6,40303,True,1,2023-03-11,NaN,NaN,NaN,0.500900
3,0,100,4,3,860,Caitlyn,51,6,96304,True,3,2023-03-11,NaN,NaN,NaN,0.485500
4,0,100,5,4,325,Morgana,25,7,608721,True,0,2023-03-11,NaN,NaN,NaN,0.501500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79983,7978,200,6,39592,630,Renekton,58,7,65103,False,0,2023-03-11,NaN,NaN,NaN,0.489350
79984,7978,200,7,39593,232,Gragas,79,7,185423,True,0,2023-03-11,NaN,NaN,NaN,0.483633
79985,7978,200,8,39594,431,Irelia,39,6,39957,False,2,2023-03-11,NaN,NaN,NaN,0.496900
79986,7978,200,9,4383,309,Xayah,498,5,25070,False,1,2023-03-11,NaN,NaN,NaN,0.504200


In [33]:
ban13_5 = pd.read_csv('../data/13_5ban_pick.csv') # between 2023-03-08 to 2023-03-22
ban13_4 = pd.read_csv('../data/13_4ban_pick.csv') # between 2023-02-23 to 2023-03-07
ban13_3 = pd.read_csv('../data/13_3ban_pick.csv') # between 2023-02-08 to 2023-02-22
ban13_1 = pd.read_csv('../data/13_1ban_pick.csv') # between 2023-01-11 to 2023-02-07

In [34]:
# Merge patch13_1 onto participants_train using championName and matchcreationDate condition
participants_train = pd.merge(participants_train, ban13_1, on=['championName'], how='left')
participants_train.loc[(participants_train['matchcreationDate'] >= '2023-01-18') & (participants_train['matchcreationDate'] <= '2023-02-07'), '13_1_pickrate'] = participants_train['pick_rate']
participants_train.drop(columns=['pick_rate'], inplace=True)

# Merge patch13_3 onto participants_train using championName and matchcreationDate condition
participants_train = pd.merge(participants_train, ban13_3, on=['championName'], how='left')
participants_train.loc[(participants_train['matchcreationDate'] >= '2023-02-08') & (participants_train['matchcreationDate'] <= '2023-02-22'), '13_3_pickrate'] = participants_train['pick_rate']
participants_train.drop(columns=['pick_rate'], inplace=True)

# Merge patch13_4 onto participants_train using championName and matchcreationDate condition
participants_train = pd.merge(participants_train, ban13_4, on=['championName'], how='left')
participants_train.loc[(participants_train['matchcreationDate'] >= '2023-02-23') & (participants_train['matchcreationDate'] <= '2023-03-07'), '13_4_pickrate'] = participants_train['pick_rate']
participants_train.drop(columns=['pick_rate'], inplace=True)

# Merge patch13_5 onto participants_train using championName and matchcreationDate condition
participants_train = pd.merge(participants_train, ban13_5, on=['championName'], how='left')
participants_train.loc[(participants_train['matchcreationDate'] >= '2023-03-08') & (participants_train['matchcreationDate'] <= '2023-03-22'), '13_5_pickrate'] = participants_train['pick_rate']
participants_train.drop(columns=['pick_rate'], inplace=True)
participants_train

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate,13_1_winrate,13_3_winrate,13_4_winrate,13_5_winrate,win_rate_x,ban_rate_x,13_1_pickrate,win_rate_y,ban_rate_y,13_3_pickrate,win_rate_x,ban_rate_x,13_4_pickrate,win_rate_y,ban_rate_y,13_5_pickrate
0,0,100,1,0,303,Mordekaiser,82,6,61890,True,3,2023-03-11,NaN,NaN,NaN,0.525100,0.51965,0.195,NaN,0.520350,0.168,NaN,0.521050,0.151,NaN,0.525100,0.149,0.046500
1,0,100,2,1,616,Sylas,517,5,37664,True,2,2023-03-11,NaN,NaN,NaN,0.490833,0.48580,0.254,NaN,0.490233,0.232,NaN,0.490633,0.213,NaN,0.490833,0.176,0.044000
2,0,100,3,2,667,Lissandra,127,6,40303,True,1,2023-03-11,NaN,NaN,NaN,0.500900,0.49860,0.008,NaN,0.495700,0.007,NaN,0.499400,0.008,NaN,0.500900,0.008,0.026000
3,0,100,4,3,860,Caitlyn,51,6,96304,True,3,2023-03-11,NaN,NaN,NaN,0.485500,0.49940,0.199,NaN,0.458300,0.209,NaN,0.498400,0.217,NaN,0.485500,0.165,0.174000
4,0,100,5,4,325,Morgana,25,7,608721,True,0,2023-03-11,NaN,NaN,NaN,0.501500,0.50570,0.290,NaN,0.506600,0.295,NaN,0.503700,0.310,NaN,0.501500,0.310,0.092000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79983,7978,200,6,39592,630,Renekton,58,7,65103,False,0,2023-03-11,NaN,NaN,NaN,0.489350,0.49615,0.022,NaN,0.493650,0.021,NaN,0.489100,0.023,NaN,0.489350,0.023,0.022000
79984,7978,200,7,39593,232,Gragas,79,7,185423,True,0,2023-03-11,NaN,NaN,NaN,0.483633,0.48365,0.005,NaN,0.481700,0.007,NaN,0.484100,0.012,NaN,0.483633,0.021,0.025667
79985,7978,200,8,39594,431,Irelia,39,6,39957,False,2,2023-03-11,NaN,NaN,NaN,0.496900,0.50160,0.209,NaN,0.498150,0.200,NaN,0.496300,0.192,NaN,0.496900,0.179,0.043500
79986,7978,200,9,4383,309,Xayah,498,5,25070,False,1,2023-03-11,NaN,NaN,NaN,0.504200,0.50880,0.013,NaN,0.510900,0.030,NaN,0.517200,0.111,NaN,0.504200,0.107,0.118000


In [35]:
# Merge patch13_1 onto participants_train using championName and matchcreationDate condition
participants_train = pd.merge(participants_train, ban13_1, on=['championName'], how='left')
participants_train.loc[(participants_train['matchcreationDate'] >= '2023-01-18') & (participants_train['matchcreationDate'] <= '2023-02-07'), '13_1_banrate'] = participants_train['ban_rate']
participants_train.drop(columns=['ban_rate'], inplace=True)

# Merge patch13_3 onto participants_train using championName and matchcreationDate condition
participants_train = pd.merge(participants_train, ban13_3, on=['championName'], how='left')
participants_train.loc[(participants_train['matchcreationDate'] >= '2023-02-08') & (participants_train['matchcreationDate'] <= '2023-02-22'), '13_3_banrate'] = participants_train['ban_rate']
participants_train.drop(columns=['ban_rate'], inplace=True)

# Merge patch13_4 onto participants_train using championName and matchcreationDate condition
participants_train = pd.merge(participants_train, ban13_4, on=['championName'], how='left')
participants_train.loc[(participants_train['matchcreationDate'] >= '2023-02-23') & (participants_train['matchcreationDate'] <= '2023-03-07'), '13_4_banrate'] = participants_train['ban_rate']
participants_train.drop(columns=['ban_rate'], inplace=True)

# Merge patch13_5 onto participants_train using championName and matchcreationDate condition
participants_train = pd.merge(participants_train, ban13_5, on=['championName'], how='left')
participants_train.loc[(participants_train['matchcreationDate'] >= '2023-03-08') & (participants_train['matchcreationDate'] <= '2023-03-22'), '13_5_banrate'] = participants_train['ban_rate']
participants_train.drop(columns=['ban_rate'], inplace=True)
participants_train

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate,13_1_winrate,13_3_winrate,13_4_winrate,13_5_winrate,win_rate_x,ban_rate_x,13_1_pickrate,win_rate_y,ban_rate_y,13_3_pickrate,win_rate_x,ban_rate_x,13_4_pickrate,win_rate_y,ban_rate_y,13_5_pickrate,win_rate_x,pick_rate_x,13_1_banrate,win_rate_y,pick_rate_y,13_3_banrate,win_rate_x,pick_rate_x,13_4_banrate,win_rate_y,pick_rate_y,13_5_banrate
0,0,100,1,0,303,Mordekaiser,82,6,61890,True,3,2023-03-11,NaN,NaN,NaN,0.525100,0.51965,0.195,NaN,0.520350,0.168,NaN,0.521050,0.151,NaN,0.525100,0.149,0.046500,0.51965,0.0545,NaN,0.520350,0.048500,NaN,0.521050,0.046500,NaN,0.525100,0.046500,0.149
1,0,100,2,1,616,Sylas,517,5,37664,True,2,2023-03-11,NaN,NaN,NaN,0.490833,0.48580,0.254,NaN,0.490233,0.232,NaN,0.490633,0.213,NaN,0.490833,0.176,0.044000,0.48580,0.0550,NaN,0.490233,0.054333,NaN,0.490633,0.048667,NaN,0.490833,0.044000,0.176
2,0,100,3,2,667,Lissandra,127,6,40303,True,1,2023-03-11,NaN,NaN,NaN,0.500900,0.49860,0.008,NaN,0.495700,0.007,NaN,0.499400,0.008,NaN,0.500900,0.008,0.026000,0.49860,0.0320,NaN,0.495700,0.026000,NaN,0.499400,0.026000,NaN,0.500900,0.026000,0.008
3,0,100,4,3,860,Caitlyn,51,6,96304,True,3,2023-03-11,NaN,NaN,NaN,0.485500,0.49940,0.199,NaN,0.458300,0.209,NaN,0.498400,0.217,NaN,0.485500,0.165,0.174000,0.49940,0.2270,NaN,0.458300,0.122500,NaN,0.498400,0.223000,NaN,0.485500,0.174000,0.165
4,0,100,5,4,325,Morgana,25,7,608721,True,0,2023-03-11,NaN,NaN,NaN,0.501500,0.50570,0.290,NaN,0.506600,0.295,NaN,0.503700,0.310,NaN,0.501500,0.310,0.092000,0.50570,0.0530,NaN,0.506600,0.097000,NaN,0.503700,0.096000,NaN,0.501500,0.092000,0.310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79983,7978,200,6,39592,630,Renekton,58,7,65103,False,0,2023-03-11,NaN,NaN,NaN,0.489350,0.49615,0.022,NaN,0.493650,0.021,NaN,0.489100,0.023,NaN,0.489350,0.023,0.022000,0.49615,0.0255,NaN,0.493650,0.024500,NaN,0.489100,0.022500,NaN,0.489350,0.022000,0.023
79984,7978,200,7,39593,232,Gragas,79,7,185423,True,0,2023-03-11,NaN,NaN,NaN,0.483633,0.48365,0.005,NaN,0.481700,0.007,NaN,0.484100,0.012,NaN,0.483633,0.021,0.025667,0.48365,0.0165,NaN,0.481700,0.020000,NaN,0.484100,0.022000,NaN,0.483633,0.025667,0.021
79985,7978,200,8,39594,431,Irelia,39,6,39957,False,2,2023-03-11,NaN,NaN,NaN,0.496900,0.50160,0.209,NaN,0.498150,0.200,NaN,0.496300,0.192,NaN,0.496900,0.179,0.043500,0.50160,0.0600,NaN,0.498150,0.052000,NaN,0.496300,0.047500,NaN,0.496900,0.043500,0.179
79986,7978,200,9,4383,309,Xayah,498,5,25070,False,1,2023-03-11,NaN,NaN,NaN,0.504200,0.50880,0.013,NaN,0.510900,0.030,NaN,0.517200,0.111,NaN,0.504200,0.107,0.118000,0.50880,0.0760,NaN,0.510900,0.101000,NaN,0.517200,0.142000,NaN,0.504200,0.118000,0.107


In [36]:
# Compute mean of winrate columns
participants_train['win_rate'] = participants_train[['13_1_winrate', '13_3_winrate', '13_4_winrate', '13_5_winrate']].mean(axis=1)

# Drop individual winrate columns
participants_train = participants_train.drop(['13_1_winrate', '13_3_winrate', '13_4_winrate', '13_5_winrate'], axis=1)
participants_train

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate,win_rate_x,ban_rate_x,13_1_pickrate,win_rate_y,ban_rate_y,13_3_pickrate,win_rate_x,ban_rate_x,13_4_pickrate,win_rate_y,ban_rate_y,13_5_pickrate,win_rate_x,pick_rate_x,13_1_banrate,win_rate_y,pick_rate_y,13_3_banrate,win_rate_x,pick_rate_x,13_4_banrate,win_rate_y,pick_rate_y,13_5_banrate,win_rate
0,0,100,1,0,303,Mordekaiser,82,6,61890,True,3,2023-03-11,0.51965,0.195,NaN,0.520350,0.168,NaN,0.521050,0.151,NaN,0.525100,0.149,0.046500,0.51965,0.0545,NaN,0.520350,0.048500,NaN,0.521050,0.046500,NaN,0.525100,0.046500,0.149,0.525100
1,0,100,2,1,616,Sylas,517,5,37664,True,2,2023-03-11,0.48580,0.254,NaN,0.490233,0.232,NaN,0.490633,0.213,NaN,0.490833,0.176,0.044000,0.48580,0.0550,NaN,0.490233,0.054333,NaN,0.490633,0.048667,NaN,0.490833,0.044000,0.176,0.490833
2,0,100,3,2,667,Lissandra,127,6,40303,True,1,2023-03-11,0.49860,0.008,NaN,0.495700,0.007,NaN,0.499400,0.008,NaN,0.500900,0.008,0.026000,0.49860,0.0320,NaN,0.495700,0.026000,NaN,0.499400,0.026000,NaN,0.500900,0.026000,0.008,0.500900
3,0,100,4,3,860,Caitlyn,51,6,96304,True,3,2023-03-11,0.49940,0.199,NaN,0.458300,0.209,NaN,0.498400,0.217,NaN,0.485500,0.165,0.174000,0.49940,0.2270,NaN,0.458300,0.122500,NaN,0.498400,0.223000,NaN,0.485500,0.174000,0.165,0.485500
4,0,100,5,4,325,Morgana,25,7,608721,True,0,2023-03-11,0.50570,0.290,NaN,0.506600,0.295,NaN,0.503700,0.310,NaN,0.501500,0.310,0.092000,0.50570,0.0530,NaN,0.506600,0.097000,NaN,0.503700,0.096000,NaN,0.501500,0.092000,0.310,0.501500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79983,7978,200,6,39592,630,Renekton,58,7,65103,False,0,2023-03-11,0.49615,0.022,NaN,0.493650,0.021,NaN,0.489100,0.023,NaN,0.489350,0.023,0.022000,0.49615,0.0255,NaN,0.493650,0.024500,NaN,0.489100,0.022500,NaN,0.489350,0.022000,0.023,0.489350
79984,7978,200,7,39593,232,Gragas,79,7,185423,True,0,2023-03-11,0.48365,0.005,NaN,0.481700,0.007,NaN,0.484100,0.012,NaN,0.483633,0.021,0.025667,0.48365,0.0165,NaN,0.481700,0.020000,NaN,0.484100,0.022000,NaN,0.483633,0.025667,0.021,0.483633
79985,7978,200,8,39594,431,Irelia,39,6,39957,False,2,2023-03-11,0.50160,0.209,NaN,0.498150,0.200,NaN,0.496300,0.192,NaN,0.496900,0.179,0.043500,0.50160,0.0600,NaN,0.498150,0.052000,NaN,0.496300,0.047500,NaN,0.496900,0.043500,0.179,0.496900
79986,7978,200,9,4383,309,Xayah,498,5,25070,False,1,2023-03-11,0.50880,0.013,NaN,0.510900,0.030,NaN,0.517200,0.111,NaN,0.504200,0.107,0.118000,0.50880,0.0760,NaN,0.510900,0.101000,NaN,0.517200,0.142000,NaN,0.504200,0.118000,0.107,0.504200


In [37]:
# Compute mean of winrate columns
participants_train['pick_rate'] = participants_train[['13_1_pickrate', '13_3_pickrate', '13_4_pickrate', '13_5_pickrate']].mean(axis=1)

# Drop individual winrate columns
participants_train = participants_train.drop(['13_1_pickrate', '13_3_pickrate', '13_4_pickrate', '13_5_pickrate'], axis=1)
participants_train

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate,win_rate_x,ban_rate_x,win_rate_y,ban_rate_y,win_rate_x,ban_rate_x,win_rate_y,ban_rate_y,win_rate_x,pick_rate_x,13_1_banrate,win_rate_y,pick_rate_y,13_3_banrate,win_rate_x,pick_rate_x,13_4_banrate,win_rate_y,pick_rate_y,13_5_banrate,win_rate,pick_rate
0,0,100,1,0,303,Mordekaiser,82,6,61890,True,3,2023-03-11,0.51965,0.195,0.520350,0.168,0.521050,0.151,0.525100,0.149,0.51965,0.0545,NaN,0.520350,0.048500,NaN,0.521050,0.046500,NaN,0.525100,0.046500,0.149,0.525100,0.046500
1,0,100,2,1,616,Sylas,517,5,37664,True,2,2023-03-11,0.48580,0.254,0.490233,0.232,0.490633,0.213,0.490833,0.176,0.48580,0.0550,NaN,0.490233,0.054333,NaN,0.490633,0.048667,NaN,0.490833,0.044000,0.176,0.490833,0.044000
2,0,100,3,2,667,Lissandra,127,6,40303,True,1,2023-03-11,0.49860,0.008,0.495700,0.007,0.499400,0.008,0.500900,0.008,0.49860,0.0320,NaN,0.495700,0.026000,NaN,0.499400,0.026000,NaN,0.500900,0.026000,0.008,0.500900,0.026000
3,0,100,4,3,860,Caitlyn,51,6,96304,True,3,2023-03-11,0.49940,0.199,0.458300,0.209,0.498400,0.217,0.485500,0.165,0.49940,0.2270,NaN,0.458300,0.122500,NaN,0.498400,0.223000,NaN,0.485500,0.174000,0.165,0.485500,0.174000
4,0,100,5,4,325,Morgana,25,7,608721,True,0,2023-03-11,0.50570,0.290,0.506600,0.295,0.503700,0.310,0.501500,0.310,0.50570,0.0530,NaN,0.506600,0.097000,NaN,0.503700,0.096000,NaN,0.501500,0.092000,0.310,0.501500,0.092000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79983,7978,200,6,39592,630,Renekton,58,7,65103,False,0,2023-03-11,0.49615,0.022,0.493650,0.021,0.489100,0.023,0.489350,0.023,0.49615,0.0255,NaN,0.493650,0.024500,NaN,0.489100,0.022500,NaN,0.489350,0.022000,0.023,0.489350,0.022000
79984,7978,200,7,39593,232,Gragas,79,7,185423,True,0,2023-03-11,0.48365,0.005,0.481700,0.007,0.484100,0.012,0.483633,0.021,0.48365,0.0165,NaN,0.481700,0.020000,NaN,0.484100,0.022000,NaN,0.483633,0.025667,0.021,0.483633,0.025667
79985,7978,200,8,39594,431,Irelia,39,6,39957,False,2,2023-03-11,0.50160,0.209,0.498150,0.200,0.496300,0.192,0.496900,0.179,0.50160,0.0600,NaN,0.498150,0.052000,NaN,0.496300,0.047500,NaN,0.496900,0.043500,0.179,0.496900,0.043500
79986,7978,200,9,4383,309,Xayah,498,5,25070,False,1,2023-03-11,0.50880,0.013,0.510900,0.030,0.517200,0.111,0.504200,0.107,0.50880,0.0760,NaN,0.510900,0.101000,NaN,0.517200,0.142000,NaN,0.504200,0.118000,0.107,0.504200,0.118000


In [38]:
# Compute mean of winrate columns
participants_train['ban_rate'] = participants_train[['13_1_banrate', '13_3_banrate', '13_4_banrate', '13_5_banrate']].mean(axis=1)

# Drop individual winrate columns
participants_train = participants_train.drop(['13_1_banrate', '13_3_banrate', '13_4_banrate', '13_5_banrate'], axis=1)
participants_train

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate,win_rate_x,ban_rate_x,win_rate_y,ban_rate_y,win_rate_x,ban_rate_x,win_rate_y,ban_rate_y,win_rate_x,pick_rate_x,win_rate_y,pick_rate_y,win_rate_x,pick_rate_x,win_rate_y,pick_rate_y,win_rate,pick_rate,ban_rate
0,0,100,1,0,303,Mordekaiser,82,6,61890,True,3,2023-03-11,0.51965,0.195,0.520350,0.168,0.521050,0.151,0.525100,0.149,0.51965,0.0545,0.520350,0.048500,0.521050,0.046500,0.525100,0.046500,0.525100,0.046500,0.149
1,0,100,2,1,616,Sylas,517,5,37664,True,2,2023-03-11,0.48580,0.254,0.490233,0.232,0.490633,0.213,0.490833,0.176,0.48580,0.0550,0.490233,0.054333,0.490633,0.048667,0.490833,0.044000,0.490833,0.044000,0.176
2,0,100,3,2,667,Lissandra,127,6,40303,True,1,2023-03-11,0.49860,0.008,0.495700,0.007,0.499400,0.008,0.500900,0.008,0.49860,0.0320,0.495700,0.026000,0.499400,0.026000,0.500900,0.026000,0.500900,0.026000,0.008
3,0,100,4,3,860,Caitlyn,51,6,96304,True,3,2023-03-11,0.49940,0.199,0.458300,0.209,0.498400,0.217,0.485500,0.165,0.49940,0.2270,0.458300,0.122500,0.498400,0.223000,0.485500,0.174000,0.485500,0.174000,0.165
4,0,100,5,4,325,Morgana,25,7,608721,True,0,2023-03-11,0.50570,0.290,0.506600,0.295,0.503700,0.310,0.501500,0.310,0.50570,0.0530,0.506600,0.097000,0.503700,0.096000,0.501500,0.092000,0.501500,0.092000,0.310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79983,7978,200,6,39592,630,Renekton,58,7,65103,False,0,2023-03-11,0.49615,0.022,0.493650,0.021,0.489100,0.023,0.489350,0.023,0.49615,0.0255,0.493650,0.024500,0.489100,0.022500,0.489350,0.022000,0.489350,0.022000,0.023
79984,7978,200,7,39593,232,Gragas,79,7,185423,True,0,2023-03-11,0.48365,0.005,0.481700,0.007,0.484100,0.012,0.483633,0.021,0.48365,0.0165,0.481700,0.020000,0.484100,0.022000,0.483633,0.025667,0.483633,0.025667,0.021
79985,7978,200,8,39594,431,Irelia,39,6,39957,False,2,2023-03-11,0.50160,0.209,0.498150,0.200,0.496300,0.192,0.496900,0.179,0.50160,0.0600,0.498150,0.052000,0.496300,0.047500,0.496900,0.043500,0.496900,0.043500,0.179
79986,7978,200,9,4383,309,Xayah,498,5,25070,False,1,2023-03-11,0.50880,0.013,0.510900,0.030,0.517200,0.111,0.504200,0.107,0.50880,0.0760,0.510900,0.101000,0.517200,0.142000,0.504200,0.118000,0.504200,0.118000,0.107


In [53]:
# Merge patch13_1 onto participants_test using championName and matchcreationDate condition
participants_test = pd.merge(participants_test, patch13_1, on=['championName'], how='left')
participants_test.loc[(participants_test['matchcreationDate'] >= '2023-01-19') & (participants_test['matchcreationDate'] <= '2023-02-07'), '13_1_winrate'] = participants_test['win_rate']
participants_test.drop(columns=['win_rate'], inplace=True)

# Merge patch13_3 onto participants_test using championName and matchcreationDate condition
participants_test = pd.merge(participants_test, patch13_3, on=['championName'], how='left')
participants_test.loc[(participants_test['matchcreationDate'] >= '2023-02-08') & (participants_test['matchcreationDate'] <= '2023-02-22'), '13_3_winrate'] = participants_test['win_rate']
participants_test.drop(columns=['win_rate'], inplace=True)

# Merge patch13_4 onto participants_test using championName and matchcreationDate condition
participants_test = pd.merge(participants_test, patch13_4, on=['championName'], how='left')
participants_test.loc[(participants_test['matchcreationDate'] >= '2023-02-23') & (participants_test['matchcreationDate'] <= '2023-03-07'), '13_4_winrate'] = participants_test['win_rate']
participants_test.drop(columns=['win_rate'], inplace=True)

# Merge patch13_5 onto participants_test using championName and matchcreationDate condition
participants_test = pd.merge(participants_test, patch13_5, on=['championName'], how='left')
participants_test.loc[(participants_test['matchcreationDate'] >= '2023-03-08') & (participants_test['matchcreationDate'] <= '2023-03-22'), '13_5_winrate'] = participants_test['win_rate']
participants_test.drop(columns=['win_rate'], inplace=True)
participants_test

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate,13_1_winrate,13_3_winrate,13_4_winrate,13_5_winrate
0,8000,100,1,23938,57,Fiora,114,3,10896,True,0,2023-03-12,NaN,NaN,NaN,0.502600
1,8000,100,2,8037,788,LeeSin,64,7,143994,False,0,2023-03-12,NaN,NaN,NaN,0.475500
2,8000,100,3,8444,566,Veigar,45,7,203012,True,0,2023-03-12,NaN,NaN,NaN,0.522467
3,8000,100,4,24831,542,Twitch,29,7,376388,True,0,2023-03-12,NaN,NaN,NaN,0.490750
4,8000,100,5,23890,224,Soraka,16,7,625009,True,0,2023-03-12,NaN,NaN,NaN,0.502100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19993,9999,200,6,45320,458,Mordekaiser,82,7,117779,True,0,2023-03-11,NaN,NaN,NaN,0.525100
19994,9999,200,7,45321,330,FiddleSticks,9,5,22922,False,0,2023-03-11,NaN,NaN,NaN,NaN
19995,9999,200,8,45322,462,Tristana,18,7,169934,False,0,2023-03-11,NaN,NaN,NaN,0.506850
19996,9999,200,9,45323,256,Samira,360,4,19716,True,0,2023-03-11,NaN,NaN,NaN,0.499000


In [54]:
# Merge patch13_1 onto participants_test using championName and matchcreationDate condition
participants_test = pd.merge(participants_test, ban13_1, on=['championName'], how='left')
participants_test.loc[(participants_test['matchcreationDate'] >= '2023-01-19') & (participants_test['matchcreationDate'] <= '2023-02-07'), '13_1_pickrate'] = participants_test['pick_rate']
participants_test.drop(columns=['pick_rate'], inplace=True)

# Merge patch13_3 onto participants_test using championName and matchcreationDate condition
participants_test = pd.merge(participants_test, ban13_3, on=['championName'], how='left')
participants_test.loc[(participants_test['matchcreationDate'] >= '2023-02-08') & (participants_test['matchcreationDate'] <= '2023-02-22'), '13_3_pickrate'] = participants_test['pick_rate']
participants_test.drop(columns=['pick_rate'], inplace=True)

# Merge patch13_4 onto participants_test using championName and matchcreationDate condition
participants_test = pd.merge(participants_test, ban13_4, on=['championName'], how='left')
participants_test.loc[(participants_test['matchcreationDate'] >= '2023-02-23') & (participants_test['matchcreationDate'] <= '2023-03-07'), '13_4_pickrate'] = participants_test['pick_rate']
participants_test.drop(columns=['pick_rate'], inplace=True)

# Merge patch13_5 onto participants_test using championName and matchcreationDate condition
participants_test = pd.merge(participants_test, ban13_5, on=['championName'], how='left')
participants_test.loc[(participants_test['matchcreationDate'] >= '2023-03-08') & (participants_test['matchcreationDate'] <= '2023-03-22'), '13_5_pickrate'] = participants_test['pick_rate']
participants_test.drop(columns=['pick_rate'], inplace=True)
participants_test

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate,13_1_winrate,13_3_winrate,13_4_winrate,13_5_winrate,win_rate_x,ban_rate_x,13_1_pickrate,win_rate_y,ban_rate_y,13_3_pickrate,win_rate_x,ban_rate_x,13_4_pickrate,win_rate_y,ban_rate_y,13_5_pickrate
0,8000,100,1,23938,57,Fiora,114,3,10896,True,0,2023-03-12,NaN,NaN,NaN,0.502600,0.503100,0.157,NaN,0.50220,0.120,NaN,0.500700,0.116,NaN,0.502600,0.110,0.054000
1,8000,100,2,8037,788,LeeSin,64,7,143994,False,0,2023-03-12,NaN,NaN,NaN,0.475500,0.457400,0.042,NaN,0.47835,0.053,NaN,0.489000,0.105,NaN,0.475500,0.131,0.077000
2,8000,100,3,8444,566,Veigar,45,7,203012,True,0,2023-03-12,NaN,NaN,NaN,0.522467,0.484500,0.024,NaN,0.48040,0.021,NaN,0.517967,0.043,NaN,0.522467,0.219,0.051333
3,8000,100,4,24831,542,Twitch,29,7,376388,True,0,2023-03-12,NaN,NaN,NaN,0.490750,0.510633,0.105,NaN,0.48725,0.127,NaN,0.506533,0.138,NaN,0.490750,0.103,0.033000
4,8000,100,5,23890,224,Soraka,16,7,625009,True,0,2023-03-12,NaN,NaN,NaN,0.502100,0.505800,0.017,NaN,0.50570,0.013,NaN,0.504100,0.014,NaN,0.502100,0.012,0.039000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19993,9999,200,6,45320,458,Mordekaiser,82,7,117779,True,0,2023-03-11,NaN,NaN,NaN,0.525100,0.519650,0.195,NaN,0.52035,0.168,NaN,0.521050,0.151,NaN,0.525100,0.149,0.046500
19994,9999,200,7,45321,330,FiddleSticks,9,5,22922,False,0,2023-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19995,9999,200,8,45322,462,Tristana,18,7,169934,False,0,2023-03-11,NaN,NaN,NaN,0.506850,0.515500,0.056,NaN,0.51315,0.054,NaN,0.506350,0.058,NaN,0.506850,0.055,0.053500
19996,9999,200,9,45323,256,Samira,360,4,19716,True,0,2023-03-11,NaN,NaN,NaN,0.499000,0.517200,0.350,NaN,0.51810,0.379,NaN,0.496300,0.310,NaN,0.499000,0.254,0.079000


In [55]:
# Merge patch13_1 onto participants_test using championName and matchcreationDate condition
participants_test = pd.merge(participants_test, ban13_1, on=['championName'], how='left')
participants_test.loc[(participants_test['matchcreationDate'] >= '2023-01-19') & (participants_test['matchcreationDate'] <= '2023-02-07'), '13_1_banrate'] = participants_test['ban_rate']
participants_test.drop(columns=['ban_rate'], inplace=True)

# Merge patch13_3 onto participants_test using championName and matchcreationDate condition
participants_test = pd.merge(participants_test, ban13_3, on=['championName'], how='left')
participants_test.loc[(participants_test['matchcreationDate'] >= '2023-02-08') & (participants_test['matchcreationDate'] <= '2023-02-22'), '13_3_banrate'] = participants_test['ban_rate']
participants_test.drop(columns=['ban_rate'], inplace=True)

# Merge patch13_4 onto participants_test using championName and matchcreationDate condition
participants_test = pd.merge(participants_test, ban13_4, on=['championName'], how='left')
participants_test.loc[(participants_test['matchcreationDate'] >= '2023-02-23') & (participants_test['matchcreationDate'] <= '2023-03-07'), '13_4_banrate'] = participants_test['ban_rate']
participants_test.drop(columns=['ban_rate'], inplace=True)

# Merge patch13_5 onto participants_test using championName and matchcreationDate condition
participants_test = pd.merge(participants_test, ban13_5, on=['championName'], how='left')
participants_test.loc[(participants_test['matchcreationDate'] >= '2023-03-08') & (participants_test['matchcreationDate'] <= '2023-03-22'), '13_5_banrate'] = participants_test['ban_rate']
participants_test.drop(columns=['ban_rate'], inplace=True)
participants_test

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate,13_1_winrate,13_3_winrate,13_4_winrate,13_5_winrate,win_rate_x,ban_rate_x,13_1_pickrate,win_rate_y,ban_rate_y,13_3_pickrate,win_rate_x,ban_rate_x,13_4_pickrate,win_rate_y,ban_rate_y,13_5_pickrate,win_rate_x,pick_rate_x,13_1_banrate,win_rate_y,pick_rate_y,13_3_banrate,win_rate_x,pick_rate_x,13_4_banrate,win_rate_y,pick_rate_y,13_5_banrate
0,8000,100,1,23938,57,Fiora,114,3,10896,True,0,2023-03-12,NaN,NaN,NaN,0.502600,0.503100,0.157,NaN,0.50220,0.120,NaN,0.500700,0.116,NaN,0.502600,0.110,0.054000,0.503100,0.065000,NaN,0.50220,0.0610,NaN,0.500700,0.058000,NaN,0.502600,0.054000,0.110
1,8000,100,2,8037,788,LeeSin,64,7,143994,False,0,2023-03-12,NaN,NaN,NaN,0.475500,0.457400,0.042,NaN,0.47835,0.053,NaN,0.489000,0.105,NaN,0.475500,0.131,0.077000,0.457400,0.082000,NaN,0.47835,0.0605,NaN,0.489000,0.134000,NaN,0.475500,0.077000,0.131
2,8000,100,3,8444,566,Veigar,45,7,203012,True,0,2023-03-12,NaN,NaN,NaN,0.522467,0.484500,0.024,NaN,0.48040,0.021,NaN,0.517967,0.043,NaN,0.522467,0.219,0.051333,0.484500,0.026500,NaN,0.48040,0.0245,NaN,0.517967,0.036333,NaN,0.522467,0.051333,0.219
3,8000,100,4,24831,542,Twitch,29,7,376388,True,0,2023-03-12,NaN,NaN,NaN,0.490750,0.510633,0.105,NaN,0.48725,0.127,NaN,0.506533,0.138,NaN,0.490750,0.103,0.033000,0.510633,0.027667,NaN,0.48725,0.0235,NaN,0.506533,0.028667,NaN,0.490750,0.033000,0.103
4,8000,100,5,23890,224,Soraka,16,7,625009,True,0,2023-03-12,NaN,NaN,NaN,0.502100,0.505800,0.017,NaN,0.50570,0.013,NaN,0.504100,0.014,NaN,0.502100,0.012,0.039000,0.505800,0.048000,NaN,0.50570,0.0430,NaN,0.504100,0.042000,NaN,0.502100,0.039000,0.012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19993,9999,200,6,45320,458,Mordekaiser,82,7,117779,True,0,2023-03-11,NaN,NaN,NaN,0.525100,0.519650,0.195,NaN,0.52035,0.168,NaN,0.521050,0.151,NaN,0.525100,0.149,0.046500,0.519650,0.054500,NaN,0.52035,0.0485,NaN,0.521050,0.046500,NaN,0.525100,0.046500,0.149
19994,9999,200,7,45321,330,FiddleSticks,9,5,22922,False,0,2023-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19995,9999,200,8,45322,462,Tristana,18,7,169934,False,0,2023-03-11,NaN,NaN,NaN,0.506850,0.515500,0.056,NaN,0.51315,0.054,NaN,0.506350,0.058,NaN,0.506850,0.055,0.053500,0.515500,0.055000,NaN,0.51315,0.0580,NaN,0.506350,0.057000,NaN,0.506850,0.053500,0.055
19996,9999,200,9,45323,256,Samira,360,4,19716,True,0,2023-03-11,NaN,NaN,NaN,0.499000,0.517200,0.350,NaN,0.51810,0.379,NaN,0.496300,0.310,NaN,0.499000,0.254,0.079000,0.517200,0.122000,NaN,0.51810,0.1150,NaN,0.496300,0.086000,NaN,0.499000,0.079000,0.254


In [56]:
# Compute mean of winrate columns
participants_test['win_rate'] = participants_test[['13_1_winrate', '13_3_winrate', '13_4_winrate', '13_5_winrate']].mean(axis=1)

# Drop individual winrate columns
participants_test = participants_test.drop(['13_1_winrate', '13_3_winrate', '13_4_winrate', '13_5_winrate'], axis=1)

In [57]:
# Compute mean of winrate columns
participants_test['pick_rate'] = participants_test[['13_1_pickrate', '13_3_pickrate', '13_4_pickrate', '13_5_pickrate']].mean(axis=1)

# Drop individual winrate columns
participants_test = participants_test.drop(['13_1_pickrate', '13_3_pickrate', '13_4_pickrate', '13_5_pickrate'], axis=1)
participants_test

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate,win_rate_x,ban_rate_x,win_rate_y,ban_rate_y,win_rate_x,ban_rate_x,win_rate_y,ban_rate_y,win_rate_x,pick_rate_x,13_1_banrate,win_rate_y,pick_rate_y,13_3_banrate,win_rate_x,pick_rate_x,13_4_banrate,win_rate_y,pick_rate_y,13_5_banrate,win_rate,pick_rate
0,8000,100,1,23938,57,Fiora,114,3,10896,True,0,2023-03-12,0.503100,0.157,0.50220,0.120,0.500700,0.116,0.502600,0.110,0.503100,0.065000,NaN,0.50220,0.0610,NaN,0.500700,0.058000,NaN,0.502600,0.054000,0.110,0.502600,0.054000
1,8000,100,2,8037,788,LeeSin,64,7,143994,False,0,2023-03-12,0.457400,0.042,0.47835,0.053,0.489000,0.105,0.475500,0.131,0.457400,0.082000,NaN,0.47835,0.0605,NaN,0.489000,0.134000,NaN,0.475500,0.077000,0.131,0.475500,0.077000
2,8000,100,3,8444,566,Veigar,45,7,203012,True,0,2023-03-12,0.484500,0.024,0.48040,0.021,0.517967,0.043,0.522467,0.219,0.484500,0.026500,NaN,0.48040,0.0245,NaN,0.517967,0.036333,NaN,0.522467,0.051333,0.219,0.522467,0.051333
3,8000,100,4,24831,542,Twitch,29,7,376388,True,0,2023-03-12,0.510633,0.105,0.48725,0.127,0.506533,0.138,0.490750,0.103,0.510633,0.027667,NaN,0.48725,0.0235,NaN,0.506533,0.028667,NaN,0.490750,0.033000,0.103,0.490750,0.033000
4,8000,100,5,23890,224,Soraka,16,7,625009,True,0,2023-03-12,0.505800,0.017,0.50570,0.013,0.504100,0.014,0.502100,0.012,0.505800,0.048000,NaN,0.50570,0.0430,NaN,0.504100,0.042000,NaN,0.502100,0.039000,0.012,0.502100,0.039000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19993,9999,200,6,45320,458,Mordekaiser,82,7,117779,True,0,2023-03-11,0.519650,0.195,0.52035,0.168,0.521050,0.151,0.525100,0.149,0.519650,0.054500,NaN,0.52035,0.0485,NaN,0.521050,0.046500,NaN,0.525100,0.046500,0.149,0.525100,0.046500
19994,9999,200,7,45321,330,FiddleSticks,9,5,22922,False,0,2023-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19995,9999,200,8,45322,462,Tristana,18,7,169934,False,0,2023-03-11,0.515500,0.056,0.51315,0.054,0.506350,0.058,0.506850,0.055,0.515500,0.055000,NaN,0.51315,0.0580,NaN,0.506350,0.057000,NaN,0.506850,0.053500,0.055,0.506850,0.053500
19996,9999,200,9,45323,256,Samira,360,4,19716,True,0,2023-03-11,0.517200,0.350,0.51810,0.379,0.496300,0.310,0.499000,0.254,0.517200,0.122000,NaN,0.51810,0.1150,NaN,0.496300,0.086000,NaN,0.499000,0.079000,0.254,0.499000,0.079000


In [58]:
# Compute mean of winrate columns
participants_test['ban_rate'] = participants_test[['13_1_banrate', '13_3_banrate', '13_4_banrate', '13_5_banrate']].mean(axis=1)

# Drop individual winrate columns
participants_test = participants_test.drop(['13_1_banrate', '13_3_banrate', '13_4_banrate', '13_5_banrate'], axis=1)
participants_test

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,matchcreationDate,win_rate_x,ban_rate_x,win_rate_y,ban_rate_y,win_rate_x,ban_rate_x,win_rate_y,ban_rate_y,win_rate_x,pick_rate_x,win_rate_y,pick_rate_y,win_rate_x,pick_rate_x,win_rate_y,pick_rate_y,win_rate,pick_rate,ban_rate
0,8000,100,1,23938,57,Fiora,114,3,10896,True,0,2023-03-12,0.503100,0.157,0.50220,0.120,0.500700,0.116,0.502600,0.110,0.503100,0.065000,0.50220,0.0610,0.500700,0.058000,0.502600,0.054000,0.502600,0.054000,0.110
1,8000,100,2,8037,788,LeeSin,64,7,143994,False,0,2023-03-12,0.457400,0.042,0.47835,0.053,0.489000,0.105,0.475500,0.131,0.457400,0.082000,0.47835,0.0605,0.489000,0.134000,0.475500,0.077000,0.475500,0.077000,0.131
2,8000,100,3,8444,566,Veigar,45,7,203012,True,0,2023-03-12,0.484500,0.024,0.48040,0.021,0.517967,0.043,0.522467,0.219,0.484500,0.026500,0.48040,0.0245,0.517967,0.036333,0.522467,0.051333,0.522467,0.051333,0.219
3,8000,100,4,24831,542,Twitch,29,7,376388,True,0,2023-03-12,0.510633,0.105,0.48725,0.127,0.506533,0.138,0.490750,0.103,0.510633,0.027667,0.48725,0.0235,0.506533,0.028667,0.490750,0.033000,0.490750,0.033000,0.103
4,8000,100,5,23890,224,Soraka,16,7,625009,True,0,2023-03-12,0.505800,0.017,0.50570,0.013,0.504100,0.014,0.502100,0.012,0.505800,0.048000,0.50570,0.0430,0.504100,0.042000,0.502100,0.039000,0.502100,0.039000,0.012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19993,9999,200,6,45320,458,Mordekaiser,82,7,117779,True,0,2023-03-11,0.519650,0.195,0.52035,0.168,0.521050,0.151,0.525100,0.149,0.519650,0.054500,0.52035,0.0485,0.521050,0.046500,0.525100,0.046500,0.525100,0.046500,0.149
19994,9999,200,7,45321,330,FiddleSticks,9,5,22922,False,0,2023-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19995,9999,200,8,45322,462,Tristana,18,7,169934,False,0,2023-03-11,0.515500,0.056,0.51315,0.054,0.506350,0.058,0.506850,0.055,0.515500,0.055000,0.51315,0.0580,0.506350,0.057000,0.506850,0.053500,0.506850,0.053500,0.055
19996,9999,200,9,45323,256,Samira,360,4,19716,True,0,2023-03-11,0.517200,0.350,0.51810,0.379,0.496300,0.310,0.499000,0.254,0.517200,0.122000,0.51810,0.1150,0.496300,0.086000,0.499000,0.079000,0.499000,0.079000,0.254


In [45]:
participants_test[participants_test['win_rate'].isna()]['championName'].values[0]

"Kai'Sa"

In [46]:
participants_train = participants_train.groupby(['matchId', 'teamId']).agg(
    champion_level=('championLevel', 'mean'),
    champion_points=('championPoints', 'mean'),
    summoner_level=('summonerLevel', 'mean'),
    chest_granted=('chestGranted', 'sum'),
    total_tokens=('tokensEarned', 'sum'),
    win_rate=('win_rate', 'mean'),
    pick_rate=('pick_rate', 'mean'),
    ban_rate=('ban_rate', 'mean')).reset_index()

participants_train

,matchId,teamId,champion_level,champion_points,summoner_level,chest_granted,total_tokens,win_rate,pick_rate,ban_rate
0,0,100,6.0,168976.4,554.2,5,9,0.500767,0.076500,0.16160
1,0,200,6.0,290062.0,417.8,5,2,0.496410,0.060400,0.09860
2,1,100,6.6,445522.0,442.6,4,2,0.505725,0.076000,0.13150
3,1,200,5.0,174316.6,317.0,3,0,0.499217,0.081033,0.11100
4,2,100,5.0,78268.0,423.8,4,2,0.506630,0.080300,0.16880
...,...,...,...,...,...,...,...,...,...,...
15995,7997,200,5.8,240754.8,211.0,2,3,0.498850,0.083000,0.07475
15996,7998,100,5.2,58050.2,376.2,3,0,0.508960,0.086900,0.03100
15997,7998,200,6.4,140731.8,570.0,4,0,0.500345,0.041100,0.07620
15998,7999,100,5.2,792441.8,492.2,4,4,0.507590,0.068300,0.08680


In [59]:
participants_test = participants_test.groupby(['matchId', 'teamId']).agg(
    champion_level=('championLevel', 'mean'),
    champion_points=('championPoints', 'mean'),
    summoner_level=('summonerLevel', 'mean'),
    chest_granted=('chestGranted', 'sum'),
    total_tokens=('tokensEarned', 'sum'),
    win_rate=('win_rate', 'mean'),
    pick_rate=('pick_rate', 'mean'),
    ban_rate=('ban_rate', 'mean')).reset_index()

participants_test

,matchId,teamId,champion_level,champion_points,summoner_level,chest_granted,total_tokens,win_rate,pick_rate,ban_rate
0,8000,100,6.2,271859.8,435.4,4,0,0.498683,0.050867,0.1150
1,8000,200,5.6,189654.8,382.8,4,2,0.487743,0.062067,0.1088
2,8001,100,6.2,229450.6,402.2,3,4,0.483420,0.080000,0.0464
3,8001,200,5.2,244007.2,296.8,5,4,0.501640,0.056200,0.0944
4,8002,100,5.8,75626.6,401.8,1,1,0.496097,0.064833,0.0954
...,...,...,...,...,...,...,...,...,...,...
3995,9997,200,5.2,403387.6,370.0,4,3,0.492830,0.095900,0.1304
3996,9998,100,6.0,239136.8,354.2,4,4,0.493780,0.028200,0.0858
3997,9998,200,6.0,214941.6,450.0,5,2,0.504127,0.079733,0.1074
3998,9999,100,5.2,55143.6,425.2,2,2,0.496840,0.069600,0.0658


In [48]:
team_train = (
    participants_train
    .pivot(index='matchId', columns='teamId')
    .reset_index()
)

team_train.columns = ['_'.join(map(str, col)).strip() for col in team_train.columns.values]
team_train = team_train.rename(columns={'matchId_': 'matchId'})
team_train

,matchId,champion_level_100,champion_level_200,champion_points_100,champion_points_200,summoner_level_100,summoner_level_200,chest_granted_100,chest_granted_200,total_tokens_100,total_tokens_200,win_rate_100,win_rate_200,pick_rate_100,pick_rate_200,ban_rate_100,ban_rate_200
0,0,6.0,6.0,168976.4,290062.0,554.2,417.8,5,5,9,2,0.500767,0.496410,0.076500,0.060400,0.16160,0.09860
1,1,6.6,5.0,445522.0,174316.6,442.6,317.0,4,3,2,0,0.505725,0.499217,0.076000,0.081033,0.13150,0.11100
2,2,5.0,4.8,78268.0,133050.2,423.8,214.0,4,2,2,0,0.506630,0.497067,0.080300,0.056933,0.16880,0.18560
3,3,5.6,6.2,128162.0,185695.4,324.8,530.2,3,3,3,2,0.502710,0.494775,0.071500,0.101250,0.07260,0.08575
4,4,5.6,4.2,129103.8,28370.2,446.4,124.0,3,4,4,6,0.502640,0.481450,0.037900,0.047900,0.05960,0.11820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,6.0,6.0,248664.4,282228.0,567.0,545.4,4,3,5,0,0.494000,0.505570,0.055500,0.028033,0.05025,0.02620
7996,7996,5.6,6.0,260344.2,111460.4,333.0,491.4,4,2,0,2,0.490467,0.495470,0.070333,0.047500,0.08720,0.05860
7997,7997,5.8,5.8,132897.0,240754.8,311.6,211.0,5,2,2,3,0.493060,0.498850,0.052200,0.083000,0.04240,0.07475
7998,7998,5.2,6.4,58050.2,140731.8,376.2,570.0,3,4,0,0,0.508960,0.500345,0.086900,0.041100,0.03100,0.07620


In [60]:
team_test = (
    participants_test
    .pivot(index='matchId', columns='teamId')
    .reset_index()
)

team_test.columns = ['_'.join(map(str, col)).strip() for col in team_test.columns.values]
team_test = team_test.rename(columns={'matchId_': 'matchId'})
team_test

,matchId,champion_level_100,champion_level_200,champion_points_100,champion_points_200,summoner_level_100,summoner_level_200,chest_granted_100,chest_granted_200,total_tokens_100,total_tokens_200,win_rate_100,win_rate_200,pick_rate_100,pick_rate_200,ban_rate_100,ban_rate_200
0,8000,6.2,5.6,271859.8,189654.8,435.4,382.8,4,4,0,2,0.498683,0.487743,0.050867,0.062067,0.1150,0.1088
1,8001,6.2,5.2,229450.6,244007.2,402.2,296.8,3,5,4,4,0.483420,0.501640,0.080000,0.056200,0.0464,0.0944
2,8002,5.8,5.0,75626.6,35481.6,401.8,445.8,1,1,1,1,0.496097,0.504930,0.064833,0.041600,0.0954,0.0466
3,8003,6.0,4.6,155956.2,20721.0,555.8,469.8,1,3,2,0,0.494395,0.498530,0.064100,0.033000,0.0672,0.0604
4,8004,5.4,5.4,224429.0,78333.6,391.2,354.2,4,4,4,2,0.501730,0.510400,0.047800,0.094500,0.0404,0.0565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9995,7.0,5.0,126858.6,45667.8,535.6,413.0,5,1,0,2,0.498050,0.492617,0.055900,0.035633,0.0360,0.0662
1996,9996,5.6,6.0,239504.0,323164.6,408.2,484.2,3,4,2,2,0.487507,0.487113,0.051233,0.069000,0.0832,0.1780
1997,9997,5.0,5.2,76340.0,403387.6,196.8,370.0,4,4,3,3,0.488575,0.492830,0.057250,0.095900,0.1440,0.1304
1998,9998,6.0,6.0,239136.8,214941.6,354.2,450.0,4,5,4,2,0.493780,0.504127,0.028200,0.079733,0.0858,0.1074


In [62]:
def extract_values(match_id):
    with open(f'../data1/timeline_{match_id}.json', 'r') as f:
        data = json.load(f)
    values = {}
    for frame in data['frames']:
        if 'events' in frame:
            for event in frame['events']:
                if event['type'] == 'CHAMPION_KILL':
                    participant_id = event['killerId']
                    if participant_id <= 5:
                        values['team100_killStreakLength'] = values.get('team100_killStreakLength', 0) + event['killStreakLength']
                    else:
                        values['team200_killStreakLength'] = values.get('team200_killStreakLength', 0) + event['killStreakLength']
                elif event['type'] == 'ELITE_MONSTER_KILL' and event['monsterType'] == 'DRAGON':
                    team_id = 'team100' if event['killerTeamId'] == 100 else 'team200'
                    values[f'{team_id}_dragonsKilled'] = values.get(f'{team_id}_dragonsKilled', 0) + 1
        if 'participantFrames' in frame:
            team100_gold = 0
            team200_gold = 0
            for participant_id, participant_frame in frame['participantFrames'].items():
                if participant_frame.get('totalGold'):
                    if int(participant_id) <= 5:
                        team100_gold += participant_frame['totalGold']
                    else:
                        team200_gold += participant_frame['totalGold']
                if participant_frame.get('jungleMinionsKilled'):
                    if int(participant_id) <= 5:
                        values['team100_jungleMinionsKilled'] = values.get('team100_jungleMinionsKilled', 0) + participant_frame['jungleMinionsKilled']
                    else:
                        values['team200_jungleMinionsKilled'] = values.get('team200_jungleMinionsKilled', 0) + participant_frame['jungleMinionsKilled']
                if participant_frame.get('minionsKilled'):
                    if int(participant_id) <= 5:
                        values['team100_minionsKilled'] = values.get('team100_minionsKilled', 0) + participant_frame['minionsKilled']
                    else:
                        values['team200_minionsKilled'] = values.get('team200_minionsKilled', 0) + participant_frame['minionsKilled']
            values['team100_totalGold'] = team100_gold
            values['team200_totalGold'] = team200_gold
            values['team100_totalDamageDoneToChampions'] = data['frames'][-1]['participantFrames']['1']['damageStats']['totalDamageDoneToChampions']
            values['team200_totalDamageDoneToChampions'] = data['frames'][-1]['participantFrames']['6']['damageStats']['totalDamageDoneToChampions']
    return values

match_values = participants_train['matchId'].apply(extract_values)
match_values_df = pd.DataFrame.from_dict(match_values.to_list())
summed_values = match_values_df.groupby(participants_train['matchId']).sum()

In [16]:
summed_values

,team100_totalGold,team200_totalGold,team100_totalDamageDoneToChampions,team200_totalDamageDoneToChampions,team100_minionsKilled,team100_jungleMinionsKilled,team200_minionsKilled,team200_jungleMinionsKilled,team200_killStreakLength,team100_killStreakLength,team200_dragonsKilled,team100_dragonsKilled
matchId,,,,,,,,,,,,
0,28292,35078,6892,8514,1884.0,526.0,2170.0,486.0,24.0,0.0,2.0,0.0
1,35324,29632,2730,8180,2032.0,606.0,2218.0,600.0,2.0,4.0,2.0,0.0
2,32748,34694,5926,5882,1782.0,632.0,1836.0,536.0,6.0,8.0,0.0,2.0
3,38494,30290,4342,6058,1868.0,516.0,1782.0,504.0,2.0,22.0,0.0,2.0
4,33552,30770,7998,5372,2104.0,660.0,2124.0,510.0,2.0,4.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,38018,29746,8388,3596,2434.0,632.0,1894.0,622.0,2.0,14.0,2.0,0.0
7996,32826,32280,11014,5294,1970.0,580.0,2050.0,568.0,2.0,8.0,2.0,0.0
7997,35490,33142,6708,6004,1698.0,304.0,1796.0,576.0,8.0,6.0,2.0,0.0


In [17]:
team_train

,matchId,champion_level_100,champion_level_200,champion_points_100,champion_points_200,summoner_level_100,summoner_level_200,chest_granted_100,chest_granted_200,total_tokens_100,total_tokens_200,win_rate_100,win_rate_200
0,0,6.0,6.0,168976.4,290062.0,554.2,417.8,5,5,9,2,0.500767,0.496410
1,1,6.6,5.0,445522.0,174316.6,442.6,317.0,4,3,2,0,0.505725,0.499217
2,2,5.0,4.8,78268.0,133050.2,423.8,214.0,4,2,2,0,0.506630,0.497067
3,3,5.6,6.2,128162.0,185695.4,324.8,530.2,3,3,3,2,0.502710,0.494775
4,4,5.6,4.2,129103.8,28370.2,446.4,124.0,3,4,4,6,0.502640,0.481450
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,6.0,6.0,248664.4,282228.0,567.0,545.4,4,3,5,0,0.494000,0.505570
7996,7996,5.6,6.0,260344.2,111460.4,333.0,491.4,4,2,0,2,0.490467,0.495470
7997,7997,5.8,5.8,132897.0,240754.8,311.6,211.0,5,2,2,3,0.493060,0.498850
7998,7998,5.2,6.4,58050.2,140731.8,376.2,570.0,3,4,0,0,0.508960,0.500345


In [63]:
team_trained = pd.merge(team_train, summed_values, on='matchId')
team_trained

,matchId,champion_level_100,champion_level_200,champion_points_100,champion_points_200,summoner_level_100,summoner_level_200,chest_granted_100,chest_granted_200,total_tokens_100,total_tokens_200,win_rate_100,win_rate_200,pick_rate_100,pick_rate_200,ban_rate_100,ban_rate_200,team100_totalGold,team200_totalGold,team100_totalDamageDoneToChampions,team200_totalDamageDoneToChampions,team100_minionsKilled,team100_jungleMinionsKilled,team200_minionsKilled,team200_jungleMinionsKilled,team200_killStreakLength,team100_killStreakLength,team200_dragonsKilled,team100_dragonsKilled
0,0,6.0,6.0,168976.4,290062.0,554.2,417.8,5,5,9,2,0.500767,0.496410,0.076500,0.060400,0.16160,0.09860,141460,175390,34460,42570,9420.0,2630.0,10850.0,2430.0,120.0,0.0,10.0,0.0
1,1,6.6,5.0,445522.0,174316.6,442.6,317.0,4,3,2,0,0.505725,0.499217,0.076000,0.081033,0.13150,0.11100,176620,148160,13650,40900,10160.0,3030.0,11090.0,3000.0,10.0,20.0,10.0,0.0
2,2,5.0,4.8,78268.0,133050.2,423.8,214.0,4,2,2,0,0.506630,0.497067,0.080300,0.056933,0.16880,0.18560,163740,173470,29630,29410,8910.0,3160.0,9180.0,2680.0,30.0,40.0,0.0,10.0
3,3,5.6,6.2,128162.0,185695.4,324.8,530.2,3,3,3,2,0.502710,0.494775,0.071500,0.101250,0.07260,0.08575,192470,151450,21710,30290,9340.0,2580.0,8910.0,2520.0,10.0,110.0,0.0,10.0
4,4,5.6,4.2,129103.8,28370.2,446.4,124.0,3,4,4,6,0.502640,0.481450,0.037900,0.047900,0.05960,0.11820,167760,153850,39990,26860,10520.0,3300.0,10620.0,2550.0,10.0,20.0,10.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,6.0,6.0,248664.4,282228.0,567.0,545.4,4,3,5,0,0.494000,0.505570,0.055500,0.028033,0.05025,0.02620,190090,148730,41940,17980,12170.0,3160.0,9470.0,3110.0,10.0,70.0,10.0,0.0
7996,7996,5.6,6.0,260344.2,111460.4,333.0,491.4,4,2,0,2,0.490467,0.495470,0.070333,0.047500,0.08720,0.05860,164130,161400,55070,26470,9850.0,2900.0,10250.0,2840.0,10.0,40.0,10.0,0.0
7997,7997,5.8,5.8,132897.0,240754.8,311.6,211.0,5,2,2,3,0.493060,0.498850,0.052200,0.083000,0.04240,0.07475,177450,165710,33540,30020,8490.0,1520.0,8980.0,2880.0,40.0,30.0,10.0,0.0
7998,7998,5.2,6.4,58050.2,140731.8,376.2,570.0,3,4,0,0,0.508960,0.500345,0.086900,0.041100,0.03100,0.07620,155430,183110,25600,80720,9530.0,2910.0,11100.0,3040.0,20.0,30.0,10.0,0.0


In [67]:
team_trained

,matchId,champion_level_100,champion_level_200,champion_points_100,champion_points_200,summoner_level_100,summoner_level_200,chest_granted_100,chest_granted_200,total_tokens_100,total_tokens_200,win_rate_100,win_rate_200,pick_rate_100,pick_rate_200,ban_rate_100,ban_rate_200,team100_totalGold,team200_totalGold,team100_totalDamageDoneToChampions,team200_totalDamageDoneToChampions,team100_minionsKilled,team100_jungleMinionsKilled,team200_minionsKilled,team200_jungleMinionsKilled,team200_killStreakLength,team100_killStreakLength,team200_dragonsKilled,team100_dragonsKilled
0,0,6.0,6.0,168976.4,290062.0,554.2,417.8,5,5,9,2,0.500767,0.496410,0.076500,0.060400,0.16160,0.09860,141460,175390,34460,42570,9420.0,2630.0,10850.0,2430.0,120.0,0.0,10.0,0.0
1,1,6.6,5.0,445522.0,174316.6,442.6,317.0,4,3,2,0,0.505725,0.499217,0.076000,0.081033,0.13150,0.11100,176620,148160,13650,40900,10160.0,3030.0,11090.0,3000.0,10.0,20.0,10.0,0.0
2,2,5.0,4.8,78268.0,133050.2,423.8,214.0,4,2,2,0,0.506630,0.497067,0.080300,0.056933,0.16880,0.18560,163740,173470,29630,29410,8910.0,3160.0,9180.0,2680.0,30.0,40.0,0.0,10.0
3,3,5.6,6.2,128162.0,185695.4,324.8,530.2,3,3,3,2,0.502710,0.494775,0.071500,0.101250,0.07260,0.08575,192470,151450,21710,30290,9340.0,2580.0,8910.0,2520.0,10.0,110.0,0.0,10.0
4,4,5.6,4.2,129103.8,28370.2,446.4,124.0,3,4,4,6,0.502640,0.481450,0.037900,0.047900,0.05960,0.11820,167760,153850,39990,26860,10520.0,3300.0,10620.0,2550.0,10.0,20.0,10.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,6.0,6.0,248664.4,282228.0,567.0,545.4,4,3,5,0,0.494000,0.505570,0.055500,0.028033,0.05025,0.02620,190090,148730,41940,17980,12170.0,3160.0,9470.0,3110.0,10.0,70.0,10.0,0.0
7996,7996,5.6,6.0,260344.2,111460.4,333.0,491.4,4,2,0,2,0.490467,0.495470,0.070333,0.047500,0.08720,0.05860,164130,161400,55070,26470,9850.0,2900.0,10250.0,2840.0,10.0,40.0,10.0,0.0
7997,7997,5.8,5.8,132897.0,240754.8,311.6,211.0,5,2,2,3,0.493060,0.498850,0.052200,0.083000,0.04240,0.07475,177450,165710,33540,30020,8490.0,1520.0,8980.0,2880.0,40.0,30.0,10.0,0.0
7998,7998,5.2,6.4,58050.2,140731.8,376.2,570.0,3,4,0,0,0.508960,0.500345,0.086900,0.041100,0.03100,0.07620,155430,183110,25600,80720,9530.0,2910.0,11100.0,3040.0,20.0,30.0,10.0,0.0


In [65]:
team_trained.columns

Index(['matchId', 'champion_level_100', 'champion_level_200',
       'champion_points_100', 'champion_points_200', 'summoner_level_100',
       'summoner_level_200', 'chest_granted_100', 'chest_granted_200',
       'total_tokens_100', 'total_tokens_200', 'win_rate_100', 'win_rate_200',
       'pick_rate_100', 'pick_rate_200', 'ban_rate_100', 'ban_rate_200',
       'team100_totalGold', 'team200_totalGold',
       'team100_totalDamageDoneToChampions',
       'team200_totalDamageDoneToChampions', 'team100_minionsKilled',
       'team100_jungleMinionsKilled', 'team200_minionsKilled',
       'team200_jungleMinionsKilled', 'team200_killStreakLength',
       'team100_killStreakLength', 'team200_dragonsKilled',
       'team100_dragonsKilled'],
      dtype='object')

In [66]:
team_tested.columns

Index(['matchId', 'champion_level_100', 'champion_level_200',
       'champion_points_100', 'champion_points_200', 'summoner_level_100',
       'summoner_level_200', 'chest_granted_100', 'chest_granted_200',
       'total_tokens_100', 'total_tokens_200', 'win_rate_100', 'win_rate_200',
       'pick_rate_100', 'pick_rate_200', 'ban_rate_100', 'ban_rate_200',
       'team100_totalGold', 'team200_totalGold',
       'team100_totalDamageDoneToChampions',
       'team200_totalDamageDoneToChampions', 'team100_minionsKilled',
       'team100_jungleMinionsKilled', 'team200_minionsKilled',
       'team200_jungleMinionsKilled', 'team100_killStreakLength',
       'team200_killStreakLength', 'team200_dragonsKilled',
       'team100_dragonsKilled'],
      dtype='object')

In [64]:
# Rod's function to extract test timelines data
def extract_values(match_id):
    with open(f'../data/timeline_{match_id}.json', 'r') as f:
        data = json.load(f)
    values = {}
    for frame in data['frames']:
        if 'events' in frame:
            for event in frame['events']:
                if event['type'] == 'CHAMPION_KILL':
                    participant_id = event['killerId']
                    if participant_id <= 5:
                        values['team100_killStreakLength'] = values.get('team100_killStreakLength', 0) + event['killStreakLength']
                    else:
                        values['team200_killStreakLength'] = values.get('team200_killStreakLength', 0) + event['killStreakLength']
                elif event['type'] == 'ELITE_MONSTER_KILL' and event['monsterType'] == 'DRAGON':
                    team_id = 'team100' if event['killerTeamId'] == 100 else 'team200'
                    values[f'{team_id}_dragonsKilled'] = values.get(f'{team_id}_dragonsKilled', 0) + 1
        if 'participantFrames' in frame:
            team100_gold = 0
            team200_gold = 0
            for participant_id, participant_frame in frame['participantFrames'].items():
                if participant_frame.get('totalGold'):
                    if int(participant_id) <= 5:
                        team100_gold += participant_frame['totalGold']
                    else:
                        team200_gold += participant_frame['totalGold']
                if participant_frame.get('jungleMinionsKilled'):
                    if int(participant_id) <= 5:
                        values['team100_jungleMinionsKilled'] = values.get('team100_jungleMinionsKilled', 0) + participant_frame['jungleMinionsKilled']
                    else:
                        values['team200_jungleMinionsKilled'] = values.get('team200_jungleMinionsKilled', 0) + participant_frame['jungleMinionsKilled']
                if participant_frame.get('minionsKilled'):
                    if int(participant_id) <= 5:
                        values['team100_minionsKilled'] = values.get('team100_minionsKilled', 0) + participant_frame['minionsKilled']
                    else:
                        values['team200_minionsKilled'] = values.get('team200_minionsKilled', 0) + participant_frame['minionsKilled']
            values['team100_totalGold'] = team100_gold
            values['team200_totalGold'] = team200_gold
            values['team100_totalDamageDoneToChampions'] = data['frames'][-1]['participantFrames']['1']['damageStats']['totalDamageDoneToChampions']
            values['team200_totalDamageDoneToChampions'] = data['frames'][-1]['participantFrames']['6']['damageStats']['totalDamageDoneToChampions']
    return values

# apply the extract_values function to each match in the train dataset and create series
match_values = team_test['matchId'].apply(extract_values)

# create df from series of dictionaries
match_values_df = pd.DataFrame.from_dict(match_values.to_list())

# group the values in the DataFrame by match ID and sum them up
summed_values = match_values_df.groupby(team_test['matchId']).sum()

# merge the test participants data with the summed up values by match ID
team_tested = pd.merge(team_test, summed_values, on='matchId')
team_tested

,matchId,champion_level_100,champion_level_200,champion_points_100,champion_points_200,summoner_level_100,summoner_level_200,chest_granted_100,chest_granted_200,total_tokens_100,total_tokens_200,win_rate_100,win_rate_200,pick_rate_100,pick_rate_200,ban_rate_100,ban_rate_200,team100_totalGold,team200_totalGold,team100_totalDamageDoneToChampions,team200_totalDamageDoneToChampions,team100_minionsKilled,team100_jungleMinionsKilled,team200_minionsKilled,team200_jungleMinionsKilled,team100_killStreakLength,team200_killStreakLength,team200_dragonsKilled,team100_dragonsKilled
0,8000,6.2,5.6,271859.8,189654.8,435.4,382.8,4,4,0,2,0.498683,0.487743,0.050867,0.062067,0.1150,0.1088,17313,16933,1159,2389,1039.0,234.0,952.0,292.0,4.0,4.0,1.0,0.0
1,8001,6.2,5.2,229450.6,244007.2,402.2,296.8,3,5,4,4,0.483420,0.501640,0.080000,0.056200,0.0464,0.0944,17492,17533,2337,815,1040.0,191.0,871.0,304.0,0.0,5.0,0.0,0.0
2,8002,5.8,5.0,75626.6,35481.6,401.8,445.8,1,1,1,1,0.496097,0.504930,0.064833,0.041600,0.0954,0.0466,16206,17480,4088,3239,759.0,284.0,766.0,196.0,3.0,5.0,0.0,0.0
3,8003,6.0,4.6,155956.2,20721.0,555.8,469.8,1,3,2,0,0.494395,0.498530,0.064100,0.033000,0.0672,0.0604,17784,18126,3777,4689,1032.0,195.0,897.0,310.0,8.0,2.0,0.0,0.0
4,8004,5.4,5.4,224429.0,78333.6,391.2,354.2,4,4,4,2,0.501730,0.510400,0.047800,0.094500,0.0404,0.0565,16341,16993,2282,1951,1101.0,160.0,949.0,232.0,0.0,3.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9995,7.0,5.0,126858.6,45667.8,535.6,413.0,5,1,0,2,0.498050,0.492617,0.055900,0.035633,0.0360,0.0662,15440,15060,4952,4053,1101.0,236.0,991.0,312.0,3.0,0.0,0.0,1.0
1996,9996,5.6,6.0,239504.0,323164.6,408.2,484.2,3,4,2,2,0.487507,0.487113,0.051233,0.069000,0.0832,0.1780,19123,17433,2961,3894,1031.0,283.0,913.0,304.0,4.0,6.0,0.0,0.0
1997,9997,5.0,5.2,76340.0,403387.6,196.8,370.0,4,4,3,3,0.488575,0.492830,0.057250,0.095900,0.1440,0.1304,14719,17129,875,4131,933.0,272.0,955.0,331.0,0.0,7.0,0.0,1.0
1998,9998,6.0,6.0,239136.8,214941.6,354.2,450.0,4,5,4,2,0.493780,0.504127,0.028200,0.079733,0.0858,0.1074,18353,15134,4264,3665,1127.0,305.0,765.0,332.0,4.0,2.0,0.0,1.0


In [22]:
X_train = team_trained.drop('matchId', axis=1)
y_train = winner_train['winner']

In [23]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [24]:
y_pred = model.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.70725


In [25]:
X_train.columns

Index(['champion_level_100', 'champion_level_200', 'champion_points_100',
       'champion_points_200', 'summoner_level_100', 'summoner_level_200',
       'chest_granted_100', 'chest_granted_200', 'total_tokens_100',
       'total_tokens_200', 'win_rate_100', 'win_rate_200', 'team100_totalGold',
       'team200_totalGold', 'team100_totalDamageDoneToChampions',
       'team200_totalDamageDoneToChampions', 'team100_minionsKilled',
       'team100_jungleMinionsKilled', 'team200_minionsKilled',
       'team200_jungleMinionsKilled', 'team200_killStreakLength',
       'team100_killStreakLength', 'team200_dragonsKilled',
       'team100_dragonsKilled'],
      dtype='object')

In [68]:
train_data = pd.merge(team_trained, winner_train, on='matchId')
train_data

,matchId,champion_level_100,champion_level_200,champion_points_100,champion_points_200,summoner_level_100,summoner_level_200,chest_granted_100,chest_granted_200,total_tokens_100,total_tokens_200,win_rate_100,win_rate_200,pick_rate_100,pick_rate_200,ban_rate_100,ban_rate_200,team100_totalGold,team200_totalGold,team100_totalDamageDoneToChampions,team200_totalDamageDoneToChampions,team100_minionsKilled,team100_jungleMinionsKilled,team200_minionsKilled,team200_jungleMinionsKilled,team200_killStreakLength,team100_killStreakLength,team200_dragonsKilled,team100_dragonsKilled,winner
0,0,6.0,6.0,168976.4,290062.0,554.2,417.8,5,5,9,2,0.500767,0.496410,0.076500,0.060400,0.16160,0.09860,141460,175390,34460,42570,9420.0,2630.0,10850.0,2430.0,120.0,0.0,10.0,0.0,200
1,1,6.6,5.0,445522.0,174316.6,442.6,317.0,4,3,2,0,0.505725,0.499217,0.076000,0.081033,0.13150,0.11100,176620,148160,13650,40900,10160.0,3030.0,11090.0,3000.0,10.0,20.0,10.0,0.0,200
2,2,5.0,4.8,78268.0,133050.2,423.8,214.0,4,2,2,0,0.506630,0.497067,0.080300,0.056933,0.16880,0.18560,163740,173470,29630,29410,8910.0,3160.0,9180.0,2680.0,30.0,40.0,0.0,10.0,200
3,3,5.6,6.2,128162.0,185695.4,324.8,530.2,3,3,3,2,0.502710,0.494775,0.071500,0.101250,0.07260,0.08575,192470,151450,21710,30290,9340.0,2580.0,8910.0,2520.0,10.0,110.0,0.0,10.0,100
4,4,5.6,4.2,129103.8,28370.2,446.4,124.0,3,4,4,6,0.502640,0.481450,0.037900,0.047900,0.05960,0.11820,167760,153850,39990,26860,10520.0,3300.0,10620.0,2550.0,10.0,20.0,10.0,0.0,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,6.0,6.0,248664.4,282228.0,567.0,545.4,4,3,5,0,0.494000,0.505570,0.055500,0.028033,0.05025,0.02620,190090,148730,41940,17980,12170.0,3160.0,9470.0,3110.0,10.0,70.0,10.0,0.0,200
7996,7996,5.6,6.0,260344.2,111460.4,333.0,491.4,4,2,0,2,0.490467,0.495470,0.070333,0.047500,0.08720,0.05860,164130,161400,55070,26470,9850.0,2900.0,10250.0,2840.0,10.0,40.0,10.0,0.0,100
7997,7997,5.8,5.8,132897.0,240754.8,311.6,211.0,5,2,2,3,0.493060,0.498850,0.052200,0.083000,0.04240,0.07475,177450,165710,33540,30020,8490.0,1520.0,8980.0,2880.0,40.0,30.0,10.0,0.0,200
7998,7998,5.2,6.4,58050.2,140731.8,376.2,570.0,3,4,0,0,0.508960,0.500345,0.086900,0.041100,0.03100,0.07620,155430,183110,25600,80720,9530.0,2910.0,11100.0,3040.0,20.0,30.0,10.0,0.0,200


In [28]:
train_data

,matchId,champion_level_100,champion_level_200,champion_points_100,champion_points_200,summoner_level_100,summoner_level_200,chest_granted_100,chest_granted_200,total_tokens_100,total_tokens_200,win_rate_100,win_rate_200,team100_totalGold,team200_totalGold,team100_totalDamageDoneToChampions,team200_totalDamageDoneToChampions,team100_minionsKilled,team100_jungleMinionsKilled,team200_minionsKilled,team200_jungleMinionsKilled,team200_killStreakLength,team100_killStreakLength,team200_dragonsKilled,team100_dragonsKilled,winner
0,0,6.0,6.0,168976.4,290062.0,554.2,417.8,5,5,9,2,0.500767,0.496410,28292,35078,6892,8514,1884.0,526.0,2170.0,486.0,24.0,0.0,2.0,0.0,200
1,1,6.6,5.0,445522.0,174316.6,442.6,317.0,4,3,2,0,0.505725,0.499217,35324,29632,2730,8180,2032.0,606.0,2218.0,600.0,2.0,4.0,2.0,0.0,200
2,2,5.0,4.8,78268.0,133050.2,423.8,214.0,4,2,2,0,0.506630,0.497067,32748,34694,5926,5882,1782.0,632.0,1836.0,536.0,6.0,8.0,0.0,2.0,200
3,3,5.6,6.2,128162.0,185695.4,324.8,530.2,3,3,3,2,0.502710,0.494775,38494,30290,4342,6058,1868.0,516.0,1782.0,504.0,2.0,22.0,0.0,2.0,100
4,4,5.6,4.2,129103.8,28370.2,446.4,124.0,3,4,4,6,0.502640,0.481450,33552,30770,7998,5372,2104.0,660.0,2124.0,510.0,2.0,4.0,2.0,0.0,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,6.0,6.0,248664.4,282228.0,567.0,545.4,4,3,5,0,0.494000,0.505570,38018,29746,8388,3596,2434.0,632.0,1894.0,622.0,2.0,14.0,2.0,0.0,200
7996,7996,5.6,6.0,260344.2,111460.4,333.0,491.4,4,2,0,2,0.490467,0.495470,32826,32280,11014,5294,1970.0,580.0,2050.0,568.0,2.0,8.0,2.0,0.0,100
7997,7997,5.8,5.8,132897.0,240754.8,311.6,211.0,5,2,2,3,0.493060,0.498850,35490,33142,6708,6004,1698.0,304.0,1796.0,576.0,8.0,6.0,2.0,0.0,200
7998,7998,5.2,6.4,58050.2,140731.8,376.2,570.0,3,4,0,0,0.508960,0.500345,31086,36622,5120,16144,1906.0,582.0,2220.0,608.0,4.0,6.0,2.0,0.0,200


In [30]:
clf1 = setup(data = train_data, target='winner')

,Description,Value
0,Session id,1016
1,Target,winner
2,Target type,Binary
3,Target mapping,"100: 0, 200: 1"
4,Original data shape,"(8000, 26)"
5,Transformed data shape,"(8000, 26)"
6,Transformed train set shape,"(5600, 26)"
7,Transformed test set shape,"(2400, 26)"
8,Numeric features,25
9,Preprocess,True


In [32]:
# compare models and select the best one
best_model = compare_models()

# predict the winners for the test data using the best model
predictions = predict_model(best_model, data=team_tested)

# generate matchId numbers from 8000 to 9999 for the test set
predictions['matchId'] = range(8000, 8000+len(team_tested), 1)
predictions = predictions[['matchId', 'prediction_label']]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.7405,0.8168,0.7316,0.7382,0.7346,0.4808,0.4812,0.0260
ridge,Ridge Classifier,0.7346,0.0000,0.7240,0.7332,0.7283,0.4690,0.4694,0.0170
gbc,Gradient Boosting Classifier,0.7318,0.8063,0.7185,0.7312,0.7246,0.4632,0.4636,0.0340
ada,Ada Boost Classifier,0.7293,0.7997,0.7164,0.7284,0.7220,0.4582,0.4587,0.0330
et,Extra Trees Classifier,0.7236,0.7977,0.7069,0.7243,0.7151,0.4467,0.4473,0.0830
lightgbm,Light Gradient Boosting Machine,0.7230,0.7970,0.7171,0.7188,0.7178,0.4459,0.4461,0.0230
rf,Random Forest Classifier,0.7227,0.7949,0.7051,0.7236,0.7140,0.4449,0.4453,0.0550
qda,Quadratic Discriminant Analysis,0.7173,0.7836,0.6935,0.7211,0.7067,0.4341,0.4348,0.0250
xgboost,Extreme Gradient Boosting,0.7173,0.7836,0.7120,0.7133,0.7123,0.4345,0.4349,0.0210
lr,Logistic Regression,0.7061,0.7748,0.6975,0.7023,0.6997,0.4119,0.4121,0.0280


In [69]:
# split the training data into features and target
X_train = train_data.drop('winner', axis=1)
#X_train = train_data.drop(['participantId', 'matchId', 'teamId', 'winner'], axis=1)
y_train = train_data['winner']

# initialize setup and setup the environment
clf1 = setup(data=train_data, target='winner')

# compare models and select the best one
best_model = compare_models()

# predict the winners for the test data using the best model
predictions = predict_model(best_model, data=team_tested)

# generate matchId numbers from 8000 to 9999 for the test set
predictions['matchId'] = range(8000, 8000+len(team_tested), 1)
predictions = predictions[['matchId', 'prediction_label']]

# rename the predicted winner column
predictions = predictions.rename(columns={'prediction_label': 'winner'})
predictions

,Description,Value
0,Session id,2190
1,Target,winner
2,Target type,Binary
3,Target mapping,"100: 0, 200: 1"
4,Original data shape,"(8000, 30)"
5,Transformed data shape,"(8000, 30)"
6,Transformed train set shape,"(5600, 30)"
7,Transformed test set shape,"(2400, 30)"
8,Numeric features,29
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.7330,0.0000,0.7175,0.7345,0.7254,0.4657,0.4665,0.0640
lda,Linear Discriminant Analysis,0.7304,0.8119,0.7178,0.7300,0.7234,0.4604,0.4611,0.0760
catboost,CatBoost Classifier,0.7271,0.7994,0.7109,0.7279,0.7190,0.4539,0.4544,1.6570
gbc,Gradient Boosting Classifier,0.7255,0.8002,0.7113,0.7253,0.7179,0.4507,0.4512,0.5170
ada,Ada Boost Classifier,0.7205,0.7933,0.7080,0.7192,0.7134,0.4408,0.4410,0.1930
et,Extra Trees Classifier,0.7139,0.7859,0.6949,0.7153,0.7045,0.4274,0.4281,0.2610
rf,Random Forest Classifier,0.7125,0.7839,0.6895,0.7153,0.7018,0.4245,0.4251,0.3630
qda,Quadratic Discriminant Analysis,0.7073,0.7723,0.7069,0.7003,0.7034,0.4146,0.4148,0.0740
lightgbm,Light Gradient Boosting Machine,0.7070,0.7836,0.6949,0.7047,0.6994,0.4136,0.4140,0.2440
lr,Logistic Regression,0.6995,0.7724,0.6895,0.6966,0.6927,0.3987,0.3991,0.6440


,matchId,winner
0,8000,100
1,8001,200
2,8002,100
3,8003,200
4,8004,100
...,...,...
1995,9995,100
1996,9996,100
1997,9997,100
1998,9998,100


In [70]:
predictions.to_csv('predictions.csv', index=False)